In [ ]:
###############################################################################
# Synthetic data generator for the Heston Stochastic Volatility Model
###############################################################################
import numpy as np
from dynamics import stoch_dyn_Heston
import os

# =============================================================================
dt = 1/252          # Time step (1 trading day)
num_trajectories = 2000   # Monte Carlo runs per initial condition
num_inits = 100           # Number of distinct initial conditions
T = 50                    # Number of time steps per trajectory (longer -> better statistics)

nx = 2  # state dimension [S, v]
nw = 2  # noise dimension

# Output directory
save_dir = "Heston/data"
os.makedirs(save_dir, exist_ok=True)

# Initial conditions grid
S0_vals = np.linspace(50, 150, int(np.sqrt(num_inits)))
v0_vals = np.linspace(0.01, 0.09, int(np.sqrt(num_inits)))
initial_conditions = np.array([[S, v] for S in S0_vals for v in v0_vals])
N = len(initial_conditions)

# Pre-allocate arrays
mean_initial = np.zeros((N, nx, 1), dtype=np.float32)
mean_final   = np.zeros((N, nx, 1), dtype=np.float32)
cov_initial  = np.zeros((N, nx, nx), dtype=np.float32)
cov_final    = np.zeros((N, nx, nx), dtype=np.float32)

# Monte Carlo Simulation
print("Generating synthetic Heston data...")

for idx, (S0, v0) in enumerate(initial_conditions):
    X  = np.zeros((num_trajectories, nx), dtype=np.float32)
    Xf = np.zeros((num_trajectories, nx), dtype=np.float32)

    for j in range(num_trajectories):
        Sk, vk = S0, v0
        for t in range(T):
            Skw, vkw = np.random.randn(), np.random.randn()
            [Xkp1, _, _] = stoch_dyn_Heston([Sk, vk, Skw, vkw])
            if t == T - 2:
                X[j, :] = [Sk, vk]
                Xf[j, :] = Xkp1
            Sk, vk = Xkp1

    # Compute means and covariances
    mean_initial[idx, :, 0] = np.mean(X, axis=0)
    mean_final[idx, :, 0]   = np.mean(Xf, axis=0)
    cov_initial[idx, :, :]  = np.cov(X.T)
    cov_final[idx, :, :]    = np.cov(Xf.T)

    if idx % 10 == 0:
        print(f"Progress: {idx}/{N} initial conditions processed...")

np.save(f"{save_dir}/mean_initial.npy", mean_initial)
np.save(f"{save_dir}/mean_final.npy", mean_final)
np.save(f"{save_dir}/cov_initial.npy", cov_initial)
np.save(f"{save_dir}/cov_final.npy", cov_final)

Generating synthetic Heston data...
Progress: 0/100 initial conditions processed...
Progress: 10/100 initial conditions processed...
Progress: 20/100 initial conditions processed...
Progress: 30/100 initial conditions processed...
Progress: 40/100 initial conditions processed...
Progress: 50/100 initial conditions processed...
Progress: 60/100 initial conditions processed...
Progress: 70/100 initial conditions processed...
Progress: 80/100 initial conditions processed...
Progress: 90/100 initial conditions processed...
